In [1]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
import yaml
from pathlib import Path
# Local imports
from ese.scripts.utils import get_option_product
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
%%yaml inference_config

log:
    root: '?'
    gether_inference_stats: '?'
    summary_compute_global_metrics: '?' 
    min_fg_pixels: 0 
    log_interval: 20 
    log_image_stats: True 
    log_pixel_stats: True 
    track_ensemble_member_scores: False 

experiment:
    exp_root: '?'
    seed: 42

data:
    # splits: ('val', 'cal')
    splits: ('val_od',)
    preload: False 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml model_cfg

# model:
#     calibrator: '?'
#     checkpoint: '?' 
#     pretrained_exp_root : None
#     _attr:
#         pretrained_select_metric: "val-dice_score"


model:
    checkpoint: '?' 
    calibrator: None
    pretrained_exp_root : None
    # pretrained_select_metric: "max-val_od-dice_score"
    _attr:
        model: 'universeg.experiment.models.QCUNet3'
        data_support_size: 64
        mix_filters: (64, 64, 64, 64)

<IPython.core.display.Javascript object>

In [4]:
%%yaml ensemble_cfg

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    member_temps: None
    member_w_metric: None
    member_paths: None
    normalize: False

<IPython.core.display.Javascript object>

In [5]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3
    square_diff: False 

global_calibration:
    num_prob_bins: 15
    num_classes: '?' 
    neighborhood_width: 3
    square_diff: False 
    loss_weights: hausdorff

<IPython.core.display.Javascript object>

In [6]:
######################################3
exp_name = '04_04_24_WBC_UniverSegInference'
group_dict = {
    "dataset": "WBC",
    "exp_group": exp_name,
    "base_models_group": "QC3Net_SupportSetAblation",
    # "calibrated_models_group": "03_30_24_RandomCircles_MassiveEnsemble_HDw_CE"
}

In [7]:
%%yaml base_cfg_args

exp_opts:
    log.gether_inference_stats: [True]
    log.summary_compute_global_metrics: [True]
    data.subsplit: [0]

submit_opts:
    seed: 42
    scratch_root: '/storage/vbutoi/scratch/UniverSeg'
    gather_sub_runs: False
    # max_ensemble_samples: 5 
    # ens_cfg_options:
    #     - ('mean', 'probs')
    #     - ('max', 'probs')
    # num_ens_membs:
    #     - 2
    #     - 4
    #     - 8
    #     - 16
    #     - 32

<IPython.core.display.Javascript object>

In [8]:
%%yaml inference_config_opts

calibrator:
    - Uncalibrated
    # - TempScaling
    # - LTS

do_ensemble:
    # - True
    - False

<IPython.core.display.Javascript object>

### Standard Calibrators.

In [9]:
inference_func = get_cal_stats
# Get the configs for the different runs.
option_set = get_ese_inference_configs(
    group_dict=group_dict,
    inf_cfg_opts=inference_config_opts,
    base_cfg_args=base_cfg_args
)

Set seed: 42


In [10]:
len(option_set)

1

In [11]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"

##################################################
with open(inf_cfg_root / "datasets" / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_inference_cfg = yaml.safe_load(file)

with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)

##################################################
base_cfg = Config(inference_config).update([calibration_cfg, model_cfg, ensemble_cfg, dataset_inference_cfg, cal_metrics_cfg])
# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

## Running Jobs

In [12]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [13]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='3',
) 

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

{'model': 'universeg.experiment.models.QCUNet3', 'data_support_size': 64, 'mix_filters': '(64, 64, 64, 64)'}
old df:         phase  epoch      loss  dice_score  \
0       train      0  0.627559    0.159469   
1      val_id      0  0.521572    0.344062   
2      val_od      0  0.443880    0.433211   
3       train      1  0.589729    0.158760   
4       train      2  0.593186    0.149580   
...       ...    ...       ...         ...   
11427   train    148  0.427082    0.381249   
11428   train    149  0.426320    0.381776   
11429   train    150  0.408885    0.396415   
11430  val_id    150  0.359940    0.558518   
11431  val_od    150  0.314878    0.586481   

                                           augmentations  \
0      [{"RandomSobelEdgesLabel": {"p": 0.5}}, {"Rand...   
1      [{"RandomSobelEdgesLabel": {"p": 0.5}}, {"Rand...   
2      [{"RandomSobelEdgesLabel": {"p": 0.5}}, {"Rand...   
3      [{"RandomSobelEdgesLabel": {"p": 0.5}}, {"Rand...   
4      [{"RandomSobelEdgesLabe

IndexError: single positional indexer is out-of-bounds

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     job_func=get_cal_stats,
#     # available_gpus=['0', '1', '2', '3'],
#     available_gpus=['0', '1', '2', '3', '4', '5', '6', '7'],
# )